# Olá, seja muito bem-vindo ao desafio de Engenheiro(a) de dados da [A3Data]

<p align="center">
  <a href="" rel="noopener">
 <img src="https://cdn-content.talentbrand.com.br/companies/394/header/ec1d053357fc5889b2daf50460552dd67b399a89.png?1606219795" alt="Project logo"></a>
</p>

### Cria um mount point no seu Google Drive para acessar os dados do DataLake

- Execute e passe o código solicitado

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


### Importação das bibliotecas necessárias para realização do exercício

In [2]:
import pandas as pd
import json
import os
import glob
from sqlalchemy import create_engine, inspect, text
from pandas import DataFrame

In [3]:
engine = create_engine('sqlite://', echo=False)

In [4]:
users_path = '/content/drive/MyDrive/datalake/landing-zone/user'

df_user = pd.DataFrame()

for filename in os.listdir(users_path):
  if filename.endswith('json'):
    temp_df = pd.read_json(os.path.join(users_path,filename))
    df_user = pd.concat([df_user,temp_df],ignore_index=True)

columns_to_keep = ['user_id','first_name','last_name','email','date_of_birth']

df_user = df_user[columns_to_keep]
df_user.to_sql('user', engine, index=False)

800

In [5]:
ecommerce_path = '/content/drive/MyDrive/datalake/landing-zone/commerce'

df_ecommerce = pd.DataFrame()

for filename in os.listdir(ecommerce_path):
  if filename.endswith('json'):
    temp_df = pd.read_json(os.path.join(ecommerce_path,filename))
    df_ecommerce = pd.concat([df_ecommerce,temp_df],ignore_index=True)

columns_to_keep = ['user_id','department','product_name','price']

df_ecommerce = df_ecommerce[columns_to_keep]
df_ecommerce.to_sql('ecommerce', engine, index=False)

800

In [6]:
device_path = '/content/drive/MyDrive/datalake/landing-zone/device'

df_device = pd.DataFrame()

for filename in os.listdir(device_path):
  if filename.endswith('json'):
    temp_df = pd.read_json(os.path.join(device_path,filename))
    df_device = pd.concat([df_device,temp_df],ignore_index=True)

columns_to_keep = ['user_id','platform','model']

df_device = df_device[columns_to_keep]
df_device.to_sql('device', engine, index=False)

800

In [7]:
connection = engine.connect()

query = text("""
SELECT *
FROM user
JOIN device ON user.user_id = device.user_id
JOIN ecommerce ON user.user_id = ecommerce.user_id
""")

result = connection.execute(query).fetchall()

connection.close()

platform_count = pd.DataFrame(result)
platform_count

,user_id,first_name,last_name,email,date_of_birth,user_id,platform,model,user_id,department,product_name,price
0,3474,Adelia,Fahey,adelia.fahey@email.com,1968-05-08,3474,Windows RT,Samsung Galaxy S7,3474,Music,Synergistic Marble Chair,53.89
1,9140,James,Simonis,james.simonis@email.com,1996-03-31,9140,Danger OS,Xiaomi Mi 8,9140,Industrial & Beauty,Small Wooden Bottle,16.24
2,1332,Lavera,Hayes,lavera.hayes@email.com,1963-03-14,1332,Windows 10 Mobile,OnePlus 5T,1332,Garden & Electronics,Awesome Linen Bench,41.64
3,2658,Weston,Jacobs,weston.jacobs@email.com,1978-05-24,2658,Windows Phone,Huawei Mate 20 Pro,2658,Movies,Lightweight Leather Clock,44.41
4,8294,Nereida,Tillman,nereida.tillman@email.com,1959-09-08,8294,Windows 10 Mobile,Samsung Galaxy S6 Edge,8294,Shoes & Computers,Small Granite Wallet,73.38
5,8294,Denny,McClure,denny.mcclure@email.com,1968-03-31,8294,Windows 10 Mobile,Samsung Galaxy S6 Edge,8294,Shoes & Computers,Small Granite Wallet,73.38


In [8]:
platform_count.to_csv('/content/drive/MyDrive/datalake/consumer-zone/plataform_count.csv', index=False)